# Selects concrete columns with a certain amount of capacity ratio

# Initialization

In [3]:
import pandas as pd
import numpy as np
import os
import sys
import comtypes.client
import math
import json
import tkinter as tk
from tkinter import filedialog
import lib.csiapi as csiapi
import lib.util as util
import configparser

%load_ext autoreload
%autoreload 2

### Connect to ETABS

In [5]:
# Initialization of ETABSObject
"""
ETABS_path_exe = r'C:/Program Files/Computers and Structures/ETABS 21_2/ETABS.exe'
if not os.path.isfile(ETABS_path_exe):
    root = tk.Tk()
    root.withdraw()
    ETABS_path = filedialog.askopenfilename(parent=root)
    root.destroy()
"""
# create API helper object
helper = comtypes.client.CreateObject("ETABSv1.Helper")
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
print("Succesfully created helper object")

try:
    #get the active ETABS object
    ETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    SapModel = ETABSObject.SapModel
    print("Succesfully created SapModel")
except (OSError, comtypes.COMError):  
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

# create folder for temporary csv files
csv_folder_path = os.getcwd() + r'\csv'
if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)

Succesfully created helper object
Succesfully created SapModel


Initialize settings and default values

In [7]:
SapModel.SetPresentUnits_2(4,6,2) # to kN, m, C

# read config files
config = configparser.ConfigParser()
config.read('config.ini')
concrete_cover = float(config['Concrete']['concrete_cover'])
distance_between_horizontal_reinforcement = float(config['Concrete']['distance_between_horizontal_reinforcement'])
distance_between_vertical_reinforcement = float(config['Concrete']['distance_between_vertical_reinforcement'])
fy = float(config['Steel']['fy'])
dt_assume = float(config['Steel']['dt_assume'])


In [8]:
SapModel.Analyze.RunAnalysis()
SapModel.DesignConcrete.StartDesign()

0

## Create groups in ETABS based on reinforcement. Optional if created manually.
cantilever_prefix must be assigned.

renew_groups can be set to False if manually generated.

In [10]:
# create groups in ETABS based on reinforcement
# Take note of renew_groups

cantilever_prefix = 'CL' # MANDATORY
beam_prefix = 'B'
girder_prefix = 'G'
girder_reinforcement = 22
cantilever_reinforcement = 19
beam_reinforcement = 19
renew_groups = False # default is False, use True if script is ran for the first time
smf_flag = True # apply rho = 1/2 and rho = 1/4 manually

cantilever_reinforcement_group = "D"+str(cantilever_reinforcement)+"_Design"
girder_reinforcement_group = "D"+str(girder_reinforcement)+"_Design"
beam_reinforcement_group = "D"+str(beam_reinforcement)+"_Design"

beam_assignments = csiapi.get_table_display(SapModel, 'Frame Assignments - Summary')

# delete previous design groups
if renew_groups:
    group_name_list = SapModel.GroupDef.GetNameList()
    for i in group_name_list[1]:
        if '_Design' in i:
            SapModel.GroupDef.Delete(i)

SapModel.GroupDef.SetGroup_1(cantilever_reinforcement_group)
SapModel.GroupDef.SetGroup_1(girder_reinforcement_group)
SapModel.GroupDef.SetGroup_1(beam_reinforcement_group)

if renew_groups:
    for j in range(len(beam_assignments)):
        if cantilever_prefix in beam_assignments['Design Section'][j]:
            SapModel.FrameObj.SetGroupAssign(str(beam_assignments['UniqueName'][j]), cantilever_reinforcement_group, False)
        elif girder_prefix in beam_assignments['Design Section'][j]:
            SapModel.FrameObj.SetGroupAssign(str(beam_assignments['UniqueName'][j]), girder_reinforcement_group, False)
        elif beam_prefix in beam_assignments['Design Section'][j]:
            SapModel.FrameObj.SetGroupAssign(str(beam_assignments['UniqueName'][j]), beam_reinforcement_group, False)


# Assign locations to beams
Each beam station is given a location ('End-I', 'Mid', 'End-J'). Cantilevers are given ('End' and 'Mid')

In [12]:
# Assinging locations to beams
concrete_design_code = SapModel.DesignConcrete.GetCode()
raw_beam_reinforcement_results = csiapi.get_table_display(SapModel, 'Concrete Beam Design Summary - ' + concrete_design_code[0])

beam_unique_name = raw_beam_reinforcement_results['UniqueName']
beam_station = raw_beam_reinforcement_results['Station']
beam_design_sect = raw_beam_reinforcement_results['DesignSect']
beam_as_top = raw_beam_reinforcement_results['AsTopTotal'] # for cantilever checking
max_station = []
min_station = []
ln = []
reinforcement_location = []
for idx in range(len(beam_unique_name)):
# for i in range(1):
    i = beam_unique_name[idx]
    local_beam_station = beam_station[idx]
    local_design_sect = beam_design_sect[idx]
    local_beam_stations = (beam_station[i == beam_unique_name]).to_numpy()
    # max_station[i] = max(local_beam_stations)
    # min_station[i] = min(local_beam_stations)
    local_max_station = max(local_beam_stations)
    local_min_station = min(local_beam_stations)
    local_ln = local_max_station - local_min_station
    max_station.append(local_max_station)
    min_station.append(local_min_station)
    # max_station.concatenate(local_max_station)
    # min_station.concatenate(local_min_station)
    ln.append(local_max_station-local_min_station)
    if cantilever_prefix in local_design_sect:
        local_beam_as_top = (beam_as_top[i == beam_unique_name]).to_numpy()
        condition1 = local_beam_as_top[0] > local_beam_as_top[-1]
        condition2 = local_beam_station <= (local_ln / 2 + local_min_station)
        # if condition1 & condition2:
        #    local_location = 'End'
        if ~(condition1 ^ condition2):
            local_location = 'End'
        else:
            local_location = 'Mid'
    elif (girder_prefix in local_design_sect) | (beam_prefix in local_design_sect):
        if local_beam_station < (1/4*local_ln + local_min_station):
            local_location = 'End-I'
        elif local_beam_station > (3/4*local_ln + local_min_station):
            local_location = 'End-J'
        else:
            local_location = 'Mid'
    reinforcement_location.append(local_location)
    # ln.concatenate(local_max_station-local_min_station)
"""
for i in beam_unique_name.unique():
    local_reinforcement_location_i = local_beam_stations < (1/4*local_ln + local_min_station)
    local_reinforcement_location_j = local_beam_stations > (3/4*local_ln + local_min_station)  
    local_reinforcement_location_m = (~(local_reinforcement_location_i | local_reinforcement_location_j))
    # local_reinforcement = local_reinforcement_location_i * 1 + local_reinforcement_location_m * 2 + local_reinforcement_location_j * 3
    # reinforcement_location.append(local_reinforcement)
    reinforcement_location = np.append(reinforcement_location, local_reinforcement)
"""
raw_beam_reinforcement_results['Location'] = reinforcement_location

### Check beam location results here

In [14]:
util.open_as_csv(raw_beam_reinforcement_results)

## Data processing & Asmin calculation

In [16]:
# Joining tables
beam_group_assignments_table = csiapi.get_table_display(SapModel, 'Group Assignments')
df = beam_group_assignments_table
df.drop(columns = ['Object Type'], inplace = True)
df = df[df['Group Name'].str.contains('_Design')]
beam_group_assignments_table = df

In [17]:
# create a table with only unique names of beam design results
beam_unique_name_table = pd.DataFrame(raw_beam_reinforcement_results['UniqueName'].unique(), columns = ['UniqueName'])

beam_assignments = csiapi.get_table_display(SapModel, 'Frame Assignments - Summary')
beam_rectangular_sections = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
concrete_materials = csiapi.get_table_display(SapModel, 'Material Properties - Concrete Data')

beam_assignments = beam_assignments[['UniqueName', 'Design Section']]
beam_rectangular_sections = beam_rectangular_sections[['Name', 'Material', 'Depth', 'Width']]
concrete_materials = concrete_materials[['Material', 'Fc']] # Fc' in kN/m^2

beam_rectangular_sections = pd.merge(beam_rectangular_sections, concrete_materials, how = 'inner')
beam_unique_name_table = pd.merge(beam_unique_name_table, beam_assignments, how = 'inner', on = 'UniqueName')
beam_unique_name_table = pd.merge(beam_unique_name_table, beam_rectangular_sections, how = 'inner', left_on = 'Design Section', right_on = 'Name', copy = False)
beam_unique_name_table = pd.merge(beam_unique_name_table, beam_group_assignments_table, how = 'inner', left_on = 'UniqueName', right_on = 'Object Unique Name')
beam_unique_name_table.drop(columns = ['Name', 'Object Unique Name'], inplace = True)
# simplify both end-I and end-J to end
as_top_end = []
as_bot_end = []
as_top_mid = []
as_bot_mid = []
# assume cover of 60 mm
rebar_size = ((beam_unique_name_table['Group Name'].str.extract('(\d+)')).astype(int))
effective_area = (beam_unique_name_table['Depth'] - concrete_cover - dt_assume / 1000 - rebar_size.iloc[:,0] / 2000) * beam_unique_name_table['Width']
as_min_coefficient = pd.DataFrame(0.25*(beam_unique_name_table['Fc']/1000)**0.5)
as_min_coefficient['coeff2'] = 1.4
as_min_coefficient = as_min_coefficient.max(axis=1) / fy # might need to reconfigure
as_min = (as_min_coefficient * effective_area)

for i in beam_unique_name_table['UniqueName']:
    local_beam_table = raw_beam_reinforcement_results[i == raw_beam_reinforcement_results['UniqueName']]
    local_as_top_end = (local_beam_table['AsTopTotal'][local_beam_table['Location'].str.contains('End')]).max()
    local_as_bot_end = (local_beam_table['AsBotTotal'][local_beam_table['Location'].str.contains('End')]).max()
    local_as_top_mid = (local_beam_table['AsTopTotal'][local_beam_table['Location'].str.contains('Mid')]).max()
    local_as_bot_mid = (local_beam_table['AsBotTotal'][local_beam_table['Location'].str.contains('Mid')]).max()
    as_top_end.append(local_as_top_end)
    as_bot_end.append(local_as_bot_end)
    as_top_mid.append(local_as_top_mid)
    as_bot_mid.append(local_as_bot_mid)
    pass
as_top_end = pd.Series(as_top_end)
as_bot_end = pd.Series(as_bot_end)
as_top_mid = pd.Series(as_top_mid)
as_bot_mid = pd.Series(as_bot_mid)
beam_unique_name_table['AsTopEnd'] = as_top_end
beam_unique_name_table['AsBotEnd'] = as_bot_end
beam_unique_name_table['AsTopMid'] = as_top_mid
beam_unique_name_table['AsBotMid'] = as_bot_mid
beam_unique_name_table['AsMin'] = as_min

beam_unique_name_table_1 = beam_unique_name_table.copy(deep = True)

as_top_end[as_top_end < as_min] = as_min[as_top_end < as_min]
as_bot_end[as_bot_end < as_min] = as_min[as_bot_end < as_min]
as_top_mid[as_top_mid < as_min] = as_min[as_top_mid < as_min]
as_bot_mid[as_bot_mid < as_min] = as_min[as_bot_mid < as_min]
# beam_unique_name_table = beam_unique_name_table
beam_unique_name_table['AsTopEnd'] = as_top_end
beam_unique_name_table['AsBotEnd'] = as_bot_end
beam_unique_name_table['AsTopMid'] = as_top_mid
beam_unique_name_table['AsBotMid'] = as_bot_mid
beam_unique_name_table['AsMin'] = as_min

beam_all_as_min = as_min > (beam_unique_name_table_1[['AsTopEnd', 'AsBotEnd', 'AsTopMid', 'AsBotMid']]).max(axis = 1)
beam_all_as_min_table = beam_unique_name_table_1[beam_all_as_min]

# Checkpoint 1: Beams with Asmin included

In [19]:
beam_all_as_min_table

,UniqueName,Design Section,Material,Depth,Width,Fc,Group Name,AsTopEnd,AsBotEnd,AsTopMid,AsBotMid,AsMin
0,354,G3060,Concrete,0.6,0.30,25000,D22_Design,0.000532,0.000532,0.000532,0.000532,0.000539
19,376,B2540,Concrete,0.4,0.25,25000,D19_Design,0.000161,0.000202,0.000000,0.000279,0.000284
116,437,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000149,0.000391,0.000000,0.000423,0.000440
117,443,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000132,0.000334,0.000000,0.000395,0.000440
118,438,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000433,0.000315,0.000149,0.000287,0.000440
119,444,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000433,0.000257,0.000139,0.000247,0.000440
121,445,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000433,0.000248,0.000157,0.000235,0.000440
122,440,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000154,0.000377,0.000000,0.000428,0.000440
123,446,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000153,0.000333,0.000002,0.000402,0.000440
129,512,B3050,Concrete,0.5,0.30,25000,D19_Design,0.000104,0.000308,0.000433,0.000249,0.000440


In [20]:
beam_unique_name_table

,UniqueName,Design Section,Material,Depth,Width,Fc,Group Name,AsTopEnd,AsBotEnd,AsTopMid,AsBotMid,AsMin
0,354,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000539,0.000539,0.000539,0.000539,0.000539
1,360,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000952,0.000661,0.000539,0.000585,0.000539
2,362,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000892,0.000703,0.000539,0.000708,0.000539
3,370,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000959,0.000709,0.000539,0.000751,0.000539
4,364,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000895,0.000846,0.000575,0.000610,0.000539
...,...,...,...,...,...,...,...,...,...,...,...,...
309,197,G5065,Concrete,0.65,0.5,25000,D22_Design,0.002779,0.001327,0.000982,0.001438,0.000982
310,182,G5065,Concrete,0.65,0.5,25000,D22_Design,0.002651,0.001290,0.000982,0.001134,0.000982
311,535,G5065,Concrete,0.65,0.5,25000,D22_Design,0.001675,0.000982,0.000982,0.000982,0.000982
312,202,G5065,Concrete,0.65,0.5,25000,D22_Design,0.002423,0.001174,0.000982,0.000983,0.000982


In [21]:
beam_unique_name_table_1

,UniqueName,Design Section,Material,Depth,Width,Fc,Group Name,AsTopEnd,AsBotEnd,AsTopMid,AsBotMid,AsMin
0,354,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000532,0.000532,0.000532,0.000532,0.000539
1,360,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000952,0.000661,0.000532,0.000585,0.000539
2,362,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000892,0.000703,0.000532,0.000708,0.000539
3,370,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000959,0.000709,0.000532,0.000751,0.000539
4,364,G3060,Concrete,0.60,0.3,25000,D22_Design,0.000895,0.000846,0.000575,0.000610,0.000539
...,...,...,...,...,...,...,...,...,...,...,...,...
309,197,G5065,Concrete,0.65,0.5,25000,D22_Design,0.002779,0.001327,0.000969,0.001438,0.000982
310,182,G5065,Concrete,0.65,0.5,25000,D22_Design,0.002651,0.001290,0.000969,0.001134,0.000982
311,535,G5065,Concrete,0.65,0.5,25000,D22_Design,0.001675,0.000969,0.000969,0.000969,0.000982
312,202,G5065,Concrete,0.65,0.5,25000,D22_Design,0.002423,0.001174,0.000969,0.000983,0.000982


# Calculate number of longitudinal reinforcement required
Results without Asmin is saved as beam_unique_name_table_1

In [23]:
# Get Asbar for each beam
rebar_area = (rebar_size/1000) ** 2 * math.pi / 4

no_rebar_required = beam_unique_name_table[['AsTopEnd','AsBotEnd','AsTopMid','AsBotMid']]/rebar_area.to_numpy()
no_rebar_required = no_rebar_required.rename(columns={'AsTopEnd': 'RebarTopEnd',
                                                      'AsBotEnd': 'RebarBotEnd',
                                                      'AsTopMid': 'RebarTopMid',
                                                      'AsBotMid': 'RebarBotMid'})

no_rebar_required_rounded = ((no_rebar_required.apply(np.ceil)).astype(int)).astype(str) # convert to int to remove decimals then convert to str

no_rebar_required_rounded.where(no_rebar_required_rounded.astype(int)>2,2, inplace = True) # minimum of 2 rein

In [24]:
df = no_rebar_required_rounded.astype(int)

In [25]:
# df[df.max(axis = 1) < df] = [df.max(axis = 1) < df]

if smf_flag:
    smf_beams = df[beam_unique_name_table['Design Section'].str.contains(girder_prefix)]
    smf_beams.loc[:,('RebarBotEnd')] = np.maximum((smf_beams.loc[:,('RebarTopEnd')]/2).apply(np.ceil),smf_beams.loc[:,('RebarBotEnd')])
    max_series = smf_beams.max(axis=1)
    max_df = max_series.to_frame()
    # Apply np.maximum to two DataFrames
    for column in smf_beams:
        smf_beams.loc[:,column] = np.maximum(np.ceil(max_series / 4), smf_beams.loc[:,column])
    df[beam_unique_name_table['Design Section'].str.contains(girder_prefix)] = smf_beams
    no_rebar_required_rounded.astype(int)

no_rebar_required_label = no_rebar_required_rounded.copy(deep = True)
no_rebar_required_rounded = no_rebar_required_rounded.astype(str)
for column in no_rebar_required_rounded:
    # print(type(column))
    no_rebar_required_label[column] = no_rebar_required_rounded[column].str.cat(rebar_size.astype(str),sep = "D")

beam_unique_name_table.join(no_rebar_required_label)

beam_sections_reinforcement = pd.concat([beam_unique_name_table[['Design Section']], no_rebar_required_label], axis = 1)
beam_sections_reinforcement_numbers = pd.concat([beam_unique_name_table[['Design Section']], no_rebar_required_rounded], axis = 1)
# beam_sections_reinforcement = beam_sections_reinforcement.merge(no_rebar_required_rounded)
beam_sections_reinforcement

unique_beam_sections_reinforcement = (beam_sections_reinforcement_numbers.drop_duplicates())

unique_beam_sections_reinforcement_count = beam_sections_reinforcement.value_counts().reset_index()

unique_beam_sections_reinforcement
# unique_beam_sections_reinforcement_count.reset_index()

,Design Section,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid
0,G3060,2,2,2,2
1,G3060,3,2,2,2
4,G3060,3,3,2,2
7,G3060,4,2,2,2
19,B2540,2,2,2,2
...,...,...,...,...,...
303,G5065,4,3,3,3
304,G5065,5,3,3,3
306,G5065,6,3,3,3
309,G5065,8,4,3,4


# Checkpoint: Unique beam reinforcing and the its count

In [27]:
util.open_as_csv(unique_beam_sections_reinforcement_count)

### Grouping parameters

In [29]:
sort_by = [1,4,2,3] # Rebar top end, bot mid, bot end, top mid
max_total_end_difference = 3 # for joints
max_total_difference = 4

sort_dict = {
    1: 'RebarTopEnd',
    2: 'RebarBotEnd',
    3: 'RebarTopMid',
    4: 'RebarBotMid'}
sort_by_string = []
for i in range(len(sort_by)):
    sort_by_string.append(sort_dict[sort_by[i]])

In [30]:
rebar_size.rename(mapper = {0: 'Rebar size'}, axis = 1, inplace = True)

no_rebar_required_and_size = pd.concat([beam_unique_name_table[['Design Section']], no_rebar_required_rounded, rebar_size], axis = 1)
# no_rebar_required_and_size.rename(columns={no_rebar_required_and_size.columns[-1]: "Rebar size" },inplace = True)

In [31]:
unique_beam_sections_reinforcement.loc[:,sort_by_string] = unique_beam_sections_reinforcement.loc[:,sort_by_string].astype(int)

unique_beam_sections_reinforcement_sorted = unique_beam_sections_reinforcement.sort_values(by = sort_by_string, ascending = False)
unique_beam_sections_reinforcement_sorted = pd.concat([unique_beam_sections_reinforcement_sorted, rebar_size], axis = 1, join = 'inner')

util.open_as_csv(unique_beam_sections_reinforcement_sorted)

unique_beam_sections_reinforcement_sorted # pairs with no_rebar_required_and_size

,Design Section,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar size
104,G6075,13,6,4,11,22
99,G6075,13,7,4,6,22
98,G6075,13,6,4,5,22
95,G6075,12,6,4,12,22
91,G6075,12,6,4,6,22
...,...,...,...,...,...,...
0,G3060,2,2,2,2,22
19,B2540,2,2,2,2,19
24,B2040,2,2,2,2,19
116,B3050,2,2,2,2,19


In [32]:
beam_unique_name_table_2 = pd.concat([beam_unique_name_table, no_rebar_required_and_size], axis = 1)

df_temp = unique_beam_sections_reinforcement_sorted.copy(deep = True)
df_temp[sort_by_string] = df_temp[sort_by_string].astype(int)
df_temp2 = beam_unique_name_table_2.copy(deep = True)
df_temp2[sort_by_string] = df_temp2[sort_by_string].astype(int)
df_temp.reset_index(inplace = True)
df_temp.drop(columns = 'index', inplace = True)
# df_temp2 = df_temp2.loc[:,~df_temp2.columns.duplicated()]

df_temp2 = df_temp2.loc[:,~df_temp2.columns.duplicated()]

#df_temp2.drop(['Material', 'Depth', 'Width', 'Fc', 'Group Name', 'AsTopEnd', 'AsBotEnd', 'AsTopMid', 'AsBotMid', 'AsMin'], axis=1, inplace = True)
df_temp2.drop(['Fc', 'Group Name', 'AsTopEnd', 'AsBotEnd', 'AsTopMid', 'AsBotMid', 'AsMin'], axis=1, inplace = True)

df_temp2.sort_values(by=sort_by_string,inplace = True, ascending = False)

beam_grouping = pd.DataFrame([])
beam_grouping_detail = pd.DataFrame([])
unique_section = df_temp2['Design Section'].unique().tolist()
unique_section_counter = np.zeros(len(unique_section), dtype='int8')

#df_temp2.sort_values(by='RebarTopEnd', ascending = False, inplace=True)
while len(df_temp2) > 0:
    df_temp.reset_index(inplace = True)
    df_temp.drop(columns = 'index', inplace = True)
    df_temp2.reset_index(inplace = True)
    df_temp2.drop(columns = 'index', inplace = True)
    
    current_beam = df_temp2.loc[0,:].transpose().copy()

    current_beam_section = current_beam['Design Section']
    unique_section_idx = unique_section.index(current_beam_section)
    unique_section_counter[unique_section_idx] += 1
    current_beam_name = current_beam_section + '-' + str(unique_section_counter[unique_section_idx])

    cond1 = current_beam['Design Section'] == df_temp2['Design Section']
    cond2 = current_beam['Rebar size'] == df_temp2['Rebar size']
    cond3 = current_beam[sort_by_string] >= df_temp2[sort_by_string]
    cond4 = current_beam[sort_by_string].sum() <= (df_temp2[sort_by_string].sum(axis=1) + max_total_difference)
    cond5 = current_beam[['RebarTopEnd','RebarBotEnd']].sum() <= (df_temp2[['RebarTopEnd','RebarBotEnd']].sum(axis=1) + max_total_end_difference)

    conddf = pd.concat([cond1, cond2, cond3, cond4, cond5],axis=1)
    allcond = conddf.all(axis=1)
    applicable_beams = df_temp2.loc[allcond,:]

    current_beam.at['Design Section'] = current_beam_name
    # current_beam['Design Section'].replace(current_beam_section, current_beam_name)
    applicable_beams.loc[:,['New Beam Label']] = current_beam_name
    # print(current_beam)
    
    beam_grouping = pd.concat([beam_grouping, applicable_beams], axis = 0)
    beam_grouping_detail = pd.concat([beam_grouping_detail,current_beam],axis=1)
    beam_grouping_detail_1 = beam_grouping_detail.transpose()
    df_temp2 = df_temp2[~allcond]

    # df_temp = df_temp.loc[1:,:]

beam_grouping_detail_1.drop(columns = 'UniqueName', inplace = True)
util.open_as_csv(beam_grouping)
util.open_as_csv(beam_grouping_detail_1)

## beam_grouping_detail_1 and beam_grouping

### Create new sections in ETABS

In [206]:
SapModel.SetModelIsLocked(False)

concrete_rectangular_table = csiapi.database_tables(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
concrete_reinforcement_table = csiapi.database_tables(SapModel, 'Frame Section Property Definitions - Concrete Beam Reinforcing')

concrete_rectangular_table.get_table_edit()
concrete_reinforcement_table.get_table_edit()

# util.open_as_csv(concrete_rectangular_table.table)

label_exists = beam_grouping_detail_1['Design Section'].isin(concrete_rectangular_table.table['Name'])
overwrite_beams = beam_grouping_detail_1.loc[label_exists]
new_beams = beam_grouping_detail_1.loc[~label_exists]

def extract_base(data_string):
    parts = data_string.split('-')
    base_string = '-'.join(parts[:-1])
    return base_string

for i in new_beams.iterrows():
    _, i = i
    beam_section = extract_base(i['Design Section'])
    old_beam_section_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == beam_section, :]
    new_beam = old_beam_section_row
    new_beam['Name'] = i['Design Section']
    new_beam['Color'] = ''
    new_beam['GUID'] = ''
    new_beam['Notes'] = ''
    concrete_rectangular_table.table = pd.concat([concrete_rectangular_table.table, new_beam], ignore_index = True, axis = 0)
    
    as_end_top = i['RebarTopEnd']*math.pi*pow(i['Rebar size']/1000,2)/4
    as_end_bot = i['RebarBotEnd']*math.pi*pow(i['Rebar size']/1000,2)/4
    old_beam_section_reinforcement = concrete_reinforcement_table.table.loc[concrete_reinforcement_table.table['Name'] == beam_section, :]
    new_beam_reinforcement = old_beam_section_reinforcement
    new_beam_reinforcement['Name'] = i['Design Section']
    new_beam_reinforcement['Top I-End Area'] = as_end_top
    new_beam_reinforcement['Top J-End Area'] = as_end_top
    new_beam_reinforcement['Bottom I-End Area'] = as_end_bot
    new_beam_reinforcement['Bottom J-End Area'] = as_end_bot
    concrete_reinforcement_table.table = pd.concat([concrete_reinforcement_table.table, new_beam_reinforcement], ignore_index = True, axis = 0)

for i in overwrite_beams.iterrows():
    _,i = i
    beam_section = extract_base(i['Design Section'])
    old_beam_coded_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == i['Design Section'], :]
    old_beam_section_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == beam_section, :]
    # print(old_beam_section_row)
    new_row = old_beam_section_row
    new_row['Name'] = i['Design Section']
    new_row['Color'] = ''
    new_row['GUID'] = ''
    new_row['Notes'] = ''
    new_row.rename(index = {new_row.index.values[0]: old_beam_coded_row.index.values[0]}, inplace = True)
    concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == i['Design Section'],:] = new_row

    as_end_top = i['RebarTopEnd']*math.pi*pow(i['Rebar size']/1000,2)/4
    as_end_bot = i['RebarBotEnd']*math.pi*pow(i['Rebar size']/1000,2)/4

    old_beam_section_reinforcement = concrete_reinforcement_table.table.loc[concrete_reinforcement_table.table['Name'] == beam_section, :]
    new_reinforcement_row = old_beam_section_reinforcement
    new_reinforcement_row['Name'] = i['Design Section']
    new_reinforcement_row['Top I-End Area'] = as_end_top
    new_reinforcement_row['Top J-End Area'] = as_end_top
    new_reinforcement_row['Bottom I-End Area'] = as_end_bot
    new_reinforcement_row['Bottom J-End Area'] = as_end_bot
    concrete_reinforcement_table.table.loc[concrete_reinforcement_table.table['Name'] == i['Design Section'],:] = new_reinforcement_row

old_beam_section_row = concrete_rectangular_table.table.loc[concrete_rectangular_table.table['Name'] == beam_section, :]

concrete_rectangular_table.set_table_edit()
concrete_reinforcement_table.set_table_edit()
SapModel.DatabaseTables.ApplyEditedTables(True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6400\3004389148.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_beam['Name'] = i['Design Section']
C:\Users\Asus\AppData\Local\Temp\ipykernel_6400\3004389148.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_beam['Color'] = ''
C:\Users\Asus\AppData\Local\Temp\ipykernel_6400\3004389148.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

[0,
 0,
 0,
 0,
 '********** DATABASE IMPORT DEFINITION DATA **********\r\n\r\nETABS Version 21.2.0 Interactive Database Import\r\nImport Type and Options:  Interactive Database Editing\r\n     Elements Deleted from DB:  Delete object from model\r\n     Other Items Deleted from DB:  Delete item from model\r\n     Elements with Same Name in Model and DB:  Replace object in model\r\n     Other Items with Same Name in Model and DB:  Replace item in model\r\n     Elements (Except Links) in the Same Location:  Allow duplicates in the same location\r\n     Link Elements in the Same Location:  Allow duplicates in the same location\r\n     Items With Same Name in a DB Table:  Overwrite previous item\r\n     Import Control for Errors:  Abort import after 20 errors\r\n     Import Control for Warnings:  Abort import after 200 warnings\r\n\r\nImport started at \r\n\r\nImport completed at 11:41:36 AM\r\nNumber of errors:  0\r\nNumber of warnings:  0\r\nNumber of informational messages:  0\r\n\r\n\r

# Assign to beams

In [209]:
beam_grouping

,UniqueName,Design Section,Material,Depth,Width,RebarTopEnd,RebarBotEnd,RebarTopMid,RebarBotMid,Rebar size,New Beam Label
0,227,G6075,Concrete,0.75,0.60,13,6,4,11,22,G6075-1
0,116,G6075,Concrete,0.75,0.60,13,7,4,6,22,G6075-2
1,104,G6075,Concrete,0.75,0.60,13,6,4,5,22,G6075-2
3,256,G6075,Concrete,0.75,0.60,12,6,4,6,22,G6075-2
4,62,G6075,Concrete,0.75,0.60,12,6,4,6,22,G6075-2
...,...,...,...,...,...,...,...,...,...,...,...
0,214,B2540,Concrete,0.40,0.25,2,2,2,3,19,B2540-1
1,198,B2540,Concrete,0.40,0.25,2,2,2,3,19,B2540-1
2,188,B2540,Concrete,0.40,0.25,2,2,2,3,19,B2540-1
3,178,B2540,Concrete,0.40,0.25,2,2,2,3,19,B2540-1


In [223]:
for i in beam_grouping.iterrows():
    _, i = i
    print(i['UniqueName'])
    SapModel.FrameObj.SetSection(str(i['UniqueName']), str(i['New Beam Label']))


227
116
104
256
62
170
243
39
68
307
66
169
406
259
118
416
319
225
286
317
168
114
254
115
60
64
53
249
102
241
103
111
108
105
110
107
310
247
250
252
245
246
112
162
113
106
161
57
40
41
56
44
312
159
160
277
258
260
43
412
451
407
413
408
253
309
58
55
262
148
150
158
348
156
347
173
46
197
182
202
189
181
536
418
255
61
414
303
67
257
117
167
63
301
36
242
371
506
37
404
534
505
279
282
308
470
472
415
417
179
507
533
504
229
355
365
357
361
324
316
213
205
216
207
211
201
230
569
579
571
553
462
363
359
209
218
577
573
575
244
561
549
563
557
559
449
405
509
372
180
508
373
374
535
510
191
200
375
199
545
520
251
518
457
195
196
454
149
210
212
377
437
443
438
444
439
445
440
446
248
512
287
297
288
300
289
302
290
304
291
305
294
306
382
143
151
144
152
145
153
146
154
184
147
157
155
86
94
87
95
88
96
89
97
90
98
91
99
92
100
93
101
327
313
314
163
164
292
293
477
469
452
473
471
356
358
315
320
206
208
165
203
166
326
570
572
298
550
299
555
194
193
455
448
383
458
450
384
366